In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='omniglot_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-3,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 50,
    "cooldown": 25,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader)

In [12]:
trainer.train()

Train Epoch: 1 [512/17352 (3%)] Loss: 1837.849731
Train Epoch: 1 [10068/17352 (58%)] Loss: 1002.983061
Train Epoch: 1 [16939/17352 (98%)] Loss: 920.359949
    epoch          : 1
    loss           : 1128.937376800312
    val_loss       : 959.5928768573973
Train Epoch: 2 [512/17352 (3%)] Loss: 917.569336
Train Epoch: 2 [10382/17352 (60%)] Loss: 851.478768
Train Epoch: 2 [17263/17352 (99%)] Loss: 868.716208
    epoch          : 2
    loss           : 884.728523550055
    val_loss       : 857.3083547773776
Train Epoch: 3 [512/17352 (3%)] Loss: 836.403992
Train Epoch: 3 [10694/17352 (62%)] Loss: 807.249934
Train Epoch: 3 [17153/17352 (99%)] Loss: 788.372303
    epoch          : 3
    loss           : 807.0321296675924
    val_loss       : 779.1332181159446
Train Epoch: 4 [512/17352 (3%)] Loss: 761.650696
Train Epoch: 4 [10007/17352 (58%)] Loss: 774.819032
Train Epoch: 4 [17049/17352 (98%)] Loss: 753.679521
    epoch          : 4
    loss           : 759.7294062491158
    val_loss       : 7